In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import numpy as np
import matplotlib.pyplot as plt

import time

In [2]:
# torch.manual_seed(42)

In [62]:
# device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(f'device: {device}')

device: cpu


In [66]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')

        self.layers = layers
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)

        self.H1 = self.linears[0]

        
    'forward pass'
    def forward(self,x,y,t):              
        
        # for i in range(len(self.layers)-2):
        #     z = self.linears[i](a)
        #     a = self.activation(z)

        a = torch.cat([x,y,t], dim = 1)    #(N,3)

        for i in range(len(self.layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)


            
        b = self.linears[-1](a) 
         
        return b
    
    # def forward_direct(self, x):
        
    #     z = x.float()
    #     H = self.linears[0].weight

    #     for i in range(len(self.layers)-2):
    #         L = self.linears[i](z)
    #         z = self.activation(L)
    #         G = (1-torch.square(z))*H.t() #\sigma'(L)*H
    #         H = torch.matmul(self.linears[i+1].weight,G.t())

    #     z = self.linears[-1](z)
         
    #     return z,H

In [67]:
# layers = np.array([2,50,50,50,50,50,1])
layers = np.array([3,20,20,20,20,20,1])
# PINN = Sequentialmodel(layers).to(device)

In [6]:
# Resetting to ensure the reported peak truly reflects the training loop, rather than including earlier setup.

# if device.type == 'cuda':
#     torch.cuda.reset_peak_memory_stats(device)

In [86]:
# Create the training data

x = torch.linspace(0,1,30, requires_grad = True).view(-1,1)
y = torch.linspace(0,1,30, requires_grad = True).view(-1,1)
t = torch.linspace(0,5,30, requires_grad = True).view(-1,1)


if torch.is_tensor(x) != True:         
    x = torch.from_numpy(x)  
if torch.is_tensor(y) != True:         
    y = torch.from_numpy(y) 
if torch.is_tensor(t) != True:         
    t = torch.from_numpy(t) 

#convert to float
x = x.float()
y = y.float()
t = t.float()

    
x_train,y_train,t_train = torch.meshgrid(x.squeeze(),y.squeeze(),t.squeeze(), indexing = 'xy')
# x_train = x_train.reshape(-1,1).to(device).requires_grad_(True)     
# y_train = y_train.reshape(-1,1).to(device).requires_grad_(True) 
# t_train = t_train.reshape(-1,1).to(device).requires_grad_(True)     

x_train = x_train.reshape(-1,1).requires_grad_(True)     # 1000000 x 1
y_train = y_train.reshape(-1,1).requires_grad_(True) 
t_train = t_train.reshape(-1,1).requires_grad_(True)     # 1000000 x 1



In [87]:
def pde_residual(x, y, t, alpha):
    u = PINN(x,y,t)

    du_dx = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    du_dy = torch.autograd.grad(u, y, torch.ones_like(u), create_graph=True)[0]
    du_dt = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0]
    du_dx_x = torch.autograd.grad(du_dx, x, torch.ones_like(du_dx), create_graph=True)[0]
    du_dy_y = torch.autograd.grad(du_dy, y, torch.ones_like(du_dy), create_graph=True)[0]

    res_pde = du_dt - alpha * (du_dx_x + du_dy_y)

    return res_pde


    

In [88]:

def initial_condition(x,y):
  u_ic = PINN(x, y, torch.zeros_like(x))
  res_ic = u_ic - ((torch.sin(np.pi * x))*(torch.sin(np.pi * y)))
  return res_ic

In [89]:
def boundary_condition(x,y,t):
    u_left = PINN(torch.full_like(t, 0),y, t)
    u_right = PINN(torch.full_like(t, 1),y, t)

    u_bottom = PINN(x,torch.full_like(t, 0), t)
    u_top = PINN(x,torch.full_like(t, 1), t)

    res_left = u_left - torch.zeros_like(t)
    res_right = u_right - torch.zeros_like(t)
    res_bottom = u_bottom - torch.zeros_like(t)
    res_top = u_top - torch.zeros_like(t)

    return res_left, res_right,res_bottom, res_top

In [83]:
def compute_losses():
   res_pde = pde_residual(x_train, y_train, t_train, alpha = 0.01) 
   res_ic = initial_condition(x_train,y_train)
   res_left, res_right,res_bottom, res_top = boundary_condition(x_train, y_train, t_train)

   loss_pde = torch.mean(res_pde**2)
   loss_ic = torch.mean(res_ic**2)
   loss_bc = torch.mean(res_left**2) + torch.mean(res_right**2) + torch.mean(res_bottom**2) + torch.mean(res_top**2)

   total_loss = loss_pde + loss_ic + loss_bc

   return total_loss



In [ ]:
# optimizer = torch.optim.Adam(PINN.parameters(), lr=0.01)

In [ ]:
# No. of epochs


# start_time = time.time()

# num_epochs = 10000



# for epoch in range(num_epochs):
#     optimizer.zero_grad()

#     total_loss = compute_losses()

    
#     total_loss.backward()

#     optimizer.step()

#     if (epoch) % 200 == 0:
#      print(f'Epoch {epoch}, Loss: {total_loss.item()}')


# end_time = time.time()

# print(f'Total Training Time: {(end_time - start_time): .4f}seconds')


    








In [90]:
torch.manual_seed(42)
# PINN = Sequentialmodel(layers).to(device)
PINN = Sequentialmodel(layers)

In [ ]:
# if device.type == 'cuda':
#     torch.cuda.reset_peak_memory_stats(device)

In [ ]:
# optimizer = torch.optim.Adam(PINN.parameters(), lr=0.01)

In [ ]:
# # Threshold loss as the stopping criteria

# max_epochs = 15000
# threshold = 0.002



# start_time = time.time()

# ep = 0
# while ep < max_epochs:
#     optimizer.zero_grad()

#     total_loss = compute_losses()

    
#     total_loss.backward()

#     optimizer.step()


#     if total_loss.item() < threshold:
#         print(f"Reached threshold loss {threshold} at epoch {ep}")
#         break

#     if (ep) % 200 == 0:
#      print(f'Epoch {ep}, Loss: {total_loss.item()}')

#     ep += 1


# print(f"Training stopped at epoch {ep}, total time {time.time() - start_time:.2f} s")





In [91]:
# Using LBFGS

optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.05,max_iter=20,history_size=50,tolerance_grad=1e-9,tolerance_change=1e-9,line_search_fn='strong_wolfe')

max_outer_steps = 15000
threshold = 0.002

start_time = time.time()
ep = 0


def closure():

    optimizer.zero_grad()
    total_loss = compute_losses()
    total_loss.backward()

    return total_loss

while ep < max_outer_steps:

    total_loss = optimizer.step(closure)

    if total_loss.item() < threshold:
        print(f"Reached threshold loss {threshold} at outer step {ep}")
        break

    if ep % 200 == 0:
        print(f'Outer {ep}, Loss: {total_loss.item()}')

    ep += 1

print(f"Training stopped at outer step {ep}, total time {time.time() - start_time:.2f} s")
    

Outer 0, Loss: 1.6748554706573486
Reached threshold loss 0.002 at outer step 15
Training stopped at outer step 15, total time 38.77 s


In [26]:
# Memory usage after training

if device.type == 'cuda':
    peak_mem = torch.cuda.max_memory_allocated(device)
    print(f'Peak GPU Memory Usage: {peak_mem / 1e6: .2f} MB')

Peak GPU Memory Usage:  32471.80 MB


In [ ]:
# Evaluate the model

x_test = torch.linspace(-1,1,130, requires_grad = False).to(device).view(-1,1)
y_test = torch.linspace(-1,1,130, requires_grad = False).to(device).view(-1,1)
t_test = torch.linspace(0,1,130, requires_grad = False).to(device).view(-1,1)

x_test,y_test,t_test = torch.meshgrid(x_test.squeeze(),y_test.squeeze(),t_test.squeeze(), indexing = 'xy')

x_test = x_test.reshape(-1,1)
y_test = y_test.reshape(-1,1)
t_test = t_test.reshape(-1,1)

PINN.eval()

with torch.no_grad():
  u_test = PINN(x_test, y_test, t_test)

# Reshape the predicted u values for contour plotting
x_test = x_test.cpu()
y_test = y_test.cpu()
t_test = t_test.cpu()

u_test = u_test.cpu()

x_test = x_test.reshape(130,130,130)
y_test = y_test.reshape(130,130,130)
t_test = t_test.reshape(130,130,130)

u_test = u_test.reshape(130,130,130)

# Plot the PINN solution as a contour plot

plt.figure(figsize=(8,6))
plt.contourf(x_test, y_test, t_test, u_test, cmap='viridis')
plt.colorbar(label='u')
plt.xlabel('x')
plt.ylabel('t')
plt.title('PINN Solution')

RuntimeError: shape '[130, 130]' is invalid for input of size 2197000